In [1]:
# Import libraries
import sys
import tweepy
import re
import pandas as pd
import unidecode
from collections import Counter

# Authenticate to Twitter API with Tweepy
consumer_key='S7e4bcPSpNhfHEeO64pjylwFC'
consumer_secret='cGLaCjvlDvfwqzkEn7hN3AkZlUMtrhLz7SFczsASnkexy10BBx'
access_key='959019738855862273-5rqxlIGK9aCvlo2z9JWahnFPcpfln2f'
access_secret='z2NQevSQfCdEp67FWYqnYd531J6KMCzp2zGjgESiP1JsX'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True) # Wait if it hits rate limit

# Read cities dataset
# We will look for a coincidence between the location of the tweet and one of the cities in the list
# If matches, we will assign Autonomous Community to tweet
df_cities = pd.read_csv('cities.csv', encoding='utf-8')

In [5]:
df_cities["city"].head()

0       Madrid
1    Barcelona
2      Sevilla
3       Bilbao
4     Valencia
Name: city, dtype: object

In [2]:
# Assign [Name, Coordinates, Radius] of each Autonomous Community]
# We will use a few AC and increase complexity in the future
Andalusia = ["Andalusia", "37.3399964,-4.5811614,250km"]
Madrid = ["Madrid", "40.5248319,-3.7715628,60km"]
Catalonia = ["Catalonia", "41.8523094,1.5745043,150km"]
Canary_Islands = ["Canary_Islands", "28.5306525,-15.7464439,400km"]
Basque_Country = ["Basque_Country", "42.9911816,-2.5543023,100km"]

CAS = [Andalusia, Madrid, Catalonia, Basque_Country, Canary_Islands]

In [20]:
# 1. Get 500 tweets (in case some of them fail to be retrieved) for each CA (500 tweets that are sent in the given radius)
# 2. Extract location of tweet. 
# 3. If location doesnt match one of the cities in the CA: ignore. Else: Save ID of the tweet's user
CAS_ids = []
for CA in CAS:
    CA_admin = CA[0].replace("_", " ")
    CA_cities = df_cities[df_cities["admin"] == CA_admin]["city"]
    tweets = tweepy.Cursor(api.search, geocode = CA[1]).items(500)  
    CA_ids = []
    for tweet in tweets:
        search_location = re.search('location\':(.+?),', str(tweet.user))
        if search_location:
            found_location = search_location.group(1)
            found_location = unidecode.unidecode(found_location) # Delete accent of location
            if any(word.lower() in found_location.lower() for word in CA_cities): # Check if city matches
                search_id = re.search('id\': (.+?),', str(tweet.user)) # Get id of tweet's user
                if search_id:
                    found_id = search_id.group(1)
                    CA_ids.append(int(found_id))
    print("All IDs obtained from {}".format(CA[0]))
    CAS_ids.append(CA_ids)


All IDs obtained from Andalusia
All IDs obtained from Madrid
All IDs obtained from Catalonia
All IDs obtained from Basque_Country
All IDs obtained from Canary_Islands


In [21]:
# Create dataframe with list of user IDs per each CA
columns = ["Andalusia", "Madrid", "Catalonia", "Basque_Country", "Canary_Islands"]
df_ids = pd.DataFrame(data=list(zip(CAS_ids[0], CAS_ids[1], CAS_ids[2], CAS_ids[3], CAS_ids[4])),columns=columns)
print(df_ids-head())

              Andalusia               Madrid            Catalonia  \
0             625707906           1098940878            372451001   
1             330958771  1141480831501897730           4142963733   
2            2358573985   917318623378788352            944384922   
3             120569754            279555269            334183160   
4             594625034   799191282186854400   901538411038875648   
5             243561380   722132324314398720              8087512   
6   1051430101135609856           1264245234  1034035283191320577   
7             149106296  1037247917055193088           1556854903   
8   1059474960480157697   900842031727218692            312858035   
9            1726179061           1263070196   798595815971422208   
10  1002267869403664384             77531723   849350355934642179   
11            383345104            236030952  1115743762062426112   
12            318605253            476173602  1076090838294413312   
13           3087482464           

In [22]:
# TODO change names of .csv to final ones
df_ids.to_csv("CAS_ids_ext2.csv", index = False)

In [3]:
df_ids = pd.read_csv("CAS_ids_ext2.csv")

In [4]:
df_ids.head()

,Andalusia,Madrid,Catalonia,Basque_Country,Canary_Islands
0,6.257079e+08,1.098941e+09,3.724510e+08,1.123535e+18,8.918831e+08
1,3.309588e+08,1.141481e+18,4.142964e+09,8.565307e+17,9.127698e+17
2,2.358574e+09,9.173186e+17,9.443849e+08,7.141839e+17,9.127698e+17
3,1.205698e+08,2.795553e+08,3.341832e+08,1.085440e+09,7.925462e+17
4,5.946250e+08,7.991913e+17,9.015384e+17,9.370071e+08,1.145663e+18


In [4]:
# For each user ID, get 100 tweets and store all of them
tweet_users_text = []
for CA in CAS:
    user_n = 1
    for user_id in df_ids[CA[0]]:
        tweet_n = 1
        user_id = str(user_id)
        try:      
            tweets = tweepy.Cursor(api.user_timeline, id=user_id).items(100)
            for tweet in tweets:
                tweet_text = [user_id, tweet.text, CA[0]]
                if not (pd.isnull(tweet.text)):
                    tweet_users_text.append(tweet_text)
                    tweet_n += 1
            print("User {} of {} in {}".format(user_n, len(df_ids[CA[0]]), CA[0]))
            user_n += 1
            
        except:
            "Ignore User"   

User 1 of 192 in Andalusia
User 2 of 192 in Andalusia
User 3 of 192 in Andalusia
User 4 of 192 in Andalusia
User 5 of 192 in Andalusia
User 6 of 192 in Andalusia
User 7 of 192 in Andalusia
User 8 of 192 in Andalusia
User 9 of 192 in Andalusia
User 10 of 192 in Andalusia
User 11 of 192 in Andalusia
User 12 of 192 in Andalusia
User 13 of 192 in Andalusia
User 14 of 192 in Andalusia
User 15 of 192 in Andalusia
User 16 of 192 in Andalusia
User 17 of 192 in Andalusia
User 18 of 192 in Andalusia
User 19 of 192 in Andalusia
User 20 of 192 in Andalusia
User 21 of 192 in Andalusia
User 22 of 192 in Andalusia
User 23 of 192 in Andalusia
User 24 of 192 in Andalusia
User 25 of 192 in Andalusia
User 26 of 192 in Andalusia
User 27 of 192 in Andalusia
User 28 of 192 in Andalusia
User 29 of 192 in Andalusia
User 30 of 192 in Andalusia
User 31 of 192 in Andalusia
User 32 of 192 in Andalusia
User 33 of 192 in Andalusia
User 34 of 192 in Andalusia
User 35 of 192 in Andalusia
User 36 of 192 in Andalusia
U

User 38 of 192 in Catalonia
User 39 of 192 in Catalonia
User 40 of 192 in Catalonia
User 41 of 192 in Catalonia
User 42 of 192 in Catalonia
User 43 of 192 in Catalonia
User 44 of 192 in Catalonia
User 45 of 192 in Catalonia
User 46 of 192 in Catalonia
User 47 of 192 in Catalonia
User 48 of 192 in Catalonia
User 49 of 192 in Catalonia
User 50 of 192 in Catalonia
User 51 of 192 in Catalonia
User 52 of 192 in Catalonia
User 53 of 192 in Catalonia
User 54 of 192 in Catalonia
User 55 of 192 in Catalonia
User 56 of 192 in Catalonia
User 57 of 192 in Catalonia
User 58 of 192 in Catalonia
User 59 of 192 in Catalonia
User 60 of 192 in Catalonia
User 61 of 192 in Catalonia
User 62 of 192 in Catalonia
User 63 of 192 in Catalonia
User 64 of 192 in Catalonia
User 65 of 192 in Catalonia
User 66 of 192 in Catalonia
User 67 of 192 in Catalonia
User 68 of 192 in Catalonia
User 69 of 192 in Catalonia
User 70 of 192 in Catalonia
User 71 of 192 in Catalonia
User 72 of 192 in Catalonia
User 73 of 192 in Ca

User 41 of 192 in Canary_Islands
User 42 of 192 in Canary_Islands
User 43 of 192 in Canary_Islands
User 44 of 192 in Canary_Islands
User 45 of 192 in Canary_Islands
User 46 of 192 in Canary_Islands
User 47 of 192 in Canary_Islands
User 48 of 192 in Canary_Islands
User 49 of 192 in Canary_Islands
User 50 of 192 in Canary_Islands
User 51 of 192 in Canary_Islands
User 52 of 192 in Canary_Islands
User 53 of 192 in Canary_Islands
User 54 of 192 in Canary_Islands
User 55 of 192 in Canary_Islands
User 56 of 192 in Canary_Islands
User 57 of 192 in Canary_Islands
User 58 of 192 in Canary_Islands
User 59 of 192 in Canary_Islands
User 60 of 192 in Canary_Islands
User 61 of 192 in Canary_Islands
User 62 of 192 in Canary_Islands
User 63 of 192 in Canary_Islands
User 64 of 192 in Canary_Islands
User 65 of 192 in Canary_Islands
User 66 of 192 in Canary_Islands
User 67 of 192 in Canary_Islands
User 68 of 192 in Canary_Islands
User 69 of 192 in Canary_Islands
User 70 of 192 in Canary_Islands
User 71 of

In [5]:
# Transform to dataframe
df_tweets = pd.DataFrame(tweet_users_text, columns = ["id", "tweet", "location"])

In [6]:
# Check number of tweets obtained. We notice that classes are imbalanced which can bias the problem
# We will analyse this later
count_CA = Counter(list(df_tweets["location"])).most_common(5)
print(count_CA)

[('Andalusia', 13684), ('Catalonia', 13600), ('Madrid', 13000), ('Basque_Country', 12256), ('Canary_Islands', 9363)]


In [7]:
# TODO change names of .csv
df_tweets.to_csv("tweet_dataset_ext2.csv", encoding='utf-8-sig', index = False)

In [ ]:
df_tweets = pd.read_csv("tweet_dataset_ext2.csv")